In [8]:
import torch
import kornia
import numpy as np
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2 as cv
from torch import autograd

In [9]:
f = 692.653256; cx = 629.321381; cy = 330.685425
p = np.array([ [692.653256, 0.000000, 629.321381, 0.000000],
[0.000000, 692.653256, 330.685425, 0.000000],
[0.000000, 0.000000, 1.000000, 0.000000] ])

In [10]:
def convert_pix_cam_coord(pix):
    px = (pix[0] - cx)/f
    py = (pix[1] - cy)/f
    return torch.FloatTensor([px,py,1])

In [11]:
def project_lid_on_img(lid_pt,R,t,p):
    lid_pt = np.append(lid_pt,[1])
    t = np.append(t,[1])
    T = np.append(R,np.array([0,0,0]).reshape(1,3),axis=0)
    T = np.append(T,t.reshape(4,1),axis=1)
    tran_pt =  np.dot(T,lid_pt)
    proj_lid_pt = np.dot(p,tran_pt).reshape(3,1)
    pix = np.array([proj_lid_pt[0]/proj_lid_pt[2],proj_lid_pt[1]/proj_lid_pt[2]]).reshape(2,1)
    return pix

In [79]:
def cost_function_trans(N, q, T, pt_m):
#     print(N.shape,q.shape,T.shape,pt_m.shape)
    R = kornia.quaternion_to_rotation_matrix(q)
    m_rotated = torch.matmul(R, pt_m)
    m_trans = torch.add(m_rotated, T)
    B = N * m_trans
    B = torch.sum(B,dim=0)
#     B = torch.mm(N,m_trans)
#     print(B.shape)
    B = B**2
    return torch.sum(B)

In [80]:
def cost_function_rot(N, q, D):
    R = kornia.quaternion_to_rotation_matrix(q)
    D_rotated = torch.matmul(R,D)
    B = N * D_rotated
    B = torch.sum(B,dim=0)
#     print("N",N)
#     print("D",D_rotated)
#     print("B",B)
    B = B**2
    return torch.sum(B)

In [95]:
cam_pnt_a = torch.FloatTensor([507,218])
cam_pnt_b = torch.FloatTensor([381,339])
cam_pnt_c = torch.FloatTensor([524,492])

N_ab = torch.cross(convert_pix_cam_coord(cam_pnt_a), convert_pix_cam_coord(cam_pnt_b)).view(3,1)
N_bc = torch.cross(convert_pix_cam_coord(cam_pnt_b), convert_pix_cam_coord(cam_pnt_c)).view(3,1)

Ncap = torch.cat((N_ab.view(3,1),N_bc.view(3,1)), dim=1)

print(N_ab,N_bc)
print(Ncap)
print(cam_pnt_b-cam_pnt_c)

tensor([[-0.1747],
        [-0.1819],
        [-0.0604]]) tensor([[-0.2209],
        [ 0.2065],
        [-0.0817]])
tensor([[-0.1747, -0.2209],
        [-0.1819,  0.2065],
        [-0.0604, -0.0817]])
tensor([-143., -153.])


In [59]:
lid_pnt_a = torch.FloatTensor([-0.475146,-0.195425,1.86017])
lid_pnt_b = torch.FloatTensor([-0.817135,0.122386,1.85952])
lid_pnt_c = torch.FloatTensor([-0.457918,0.543083,1.83382])

# R_temp = torch.FloatTensor([[0.866,0.5,0],[-0.5,0.866,0],[0,0,1]])

# lid_pnt_a = torch.mm(R_temp, lid_pnt_a.view(3,1))
# lid_pnt_b = torch.mm(R_temp, lid_pnt_b.view(3,1))
# lid_pnt_c = torch.mm(R_temp, lid_pnt_c.view(3,1))

dir_ab = lid_pnt_a - lid_pnt_b
dir_cb = lid_pnt_c - lid_pnt_b

dir = torch.cat((dir_ab.view(3,1), dir_cb.view(3,1)), dim=1)
print(dir_ab, dir_cb)
print(dir)

tensor([[ 0.1373],
        [-0.4462],
        [ 0.0007]]) tensor([[ 0.5214],
        [ 0.1847],
        [-0.0257]])
tensor([[ 0.1373,  0.5214],
        [-0.4462,  0.1847],
        [ 0.0007, -0.0257]])


In [60]:
q_init = np.array([0,0,0,1])
T_init = np.array([0,0,0])
q = autograd.Variable(torch.FloatTensor([0,0,0,1]),requires_grad=True)
T = autograd.Variable(torch.FloatTensor([0,0,0]),requires_grad=True)
# D = autograd.Variable(dir_cb.view(3,1))
# N = autograd.Variable(N_bc)
D = autograd.Variable(dir)
N = autograd.Variable(Ncap)

In [61]:
total = autograd.Variable(torch.FloatTensor([0]))

In [62]:
for i in range(1000):
    total = cost_function_rot(N, q, D)
    total.backward()
    q.data -= .1 * q.grad.data
    q.data = kornia.normalize_quaternion(q.data)
    q.grad.data.zero_()
print(total,q.data)

tensor(3.4518e-06, grad_fn=<SumBackward0>) tensor([-0.0549, -0.0077,  0.2310,  0.9714])


In [63]:
R_final = kornia.quaternion_to_rotation_matrix(q.data).numpy()

In [64]:
R_final

tensor([0., 0., 0.])

In [88]:
lid_pts = torch.cat((lid_pnt_a.view(3,1),lid_pnt_c.view(3,1)), dim=1)
for i in range(10000):
    total_trans = cost_function_trans(N, q, T.view(3,1), lid_pts)
    total_trans.backward()
    T.data -= .01 * T.grad.data
    T.grad.data.zero_()
print(total_trans,T.data)

tensor(1.0651e-11, grad_fn=<SumBackward0>) tensor([ 0.2158, -0.3330,  0.0814])


In [91]:
pix_a = project_lid_on_img(lid_pnt_a,R_final,T.data.numpy(),p)
pix_b = project_lid_on_img(lid_pnt_b,R_final,T.data.numpy(),p)
pix_c = project_lid_on_img(lid_pnt_c,R_final,T.data.numpy(),p)
# pix_a = project_lid_on_img(lid_pnt_a,np.eye(3),np.zeros(3),p)
# pix_b = project_lid_on_img(lid_pnt_b,np.eye(3),np.zeros(3),p)
# pix_c = project_lid_on_img(lid_pnt_c,np.eye(3),np.zeros(3),p)

print(pix_a,pix_b,pix_c)

[[505.46985315]
 [219.47562968]] [[383.45211167]
 [339.7583912 ]] [[522.06077238]
 [489.92664123]]


In [92]:
img = cv.imread("/home/chinmay/Downloads/test_image.png")
img.shape

(720, 1280, 3)

In [93]:
img_circled = cv.circle(img, tuple(pix_a), 4, (255,0,0), thickness=2, lineType=8, shift=0)
img_circled = cv.circle(img, tuple(pix_b), 4, (255,0,0), thickness=2, lineType=8, shift=0)
img_circled = cv.circle(img, tuple(pix_c), 4, (255,0,0), thickness=2, lineType=8, shift=0)

In [94]:
# cv.imwrite("/home/chinmay/Desktop/output_ml_30_rnt.png",img_circled)
cv.imshow('Transformed lidar pnts',img_circled)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
torch.sum(c,dim=0)
def project_lid_on_img(lid_pt,R,t,p):
    lid_pt = np.append(lid_pt,[1])
    t = np.append(t,[1])
    T = np.append(R,np.array([0,0,0]).reshape(1,3),axis=0)
    T = np.append(T,t.reshape(4,1),axis=1)
    tran_pt =  np.dot(T,lid_pt)
    proj_lid_pt = np.dot(p,tran_pt).reshape(3,1)
    pix = np.array([proj_lid_pt[0]/proj_lid_pt[2],proj_lid_pt[1]/proj_lid_pt[2]]).reshape(2,1)
    return pix

In [ ]:
def project_lid_on_img_1(lid_pt,R,t,p):
    lid_pt = torch.cat((torch.from_numpy(lid_pt).float(),torch.FloatTensor([1])))
    t = torch.cat((t.float(),torch.FloatTensor([1])))
    T = torch.cat((torch.from_numpy(R).float(),torch.FloatTensor([0,0,0]).view(1,3)))
    T = torch.cat((T,t.view(4,1)),dim=1)
    print(T.shape,lid_pt.shape)
    tran_pt =  torch.mm(T,lid_pt.view(4,1))
    proj_lid_pt = torch.mm(torch.FloatTensor(p),tran_pt).view(3,1)
    pix = torch.FloatTensor([proj_lid_pt[0]/proj_lid_pt[2],proj_lid_pt[1]/proj_lid_pt[2]]).reshape(2,1)
    return pix

mid_img_ab = (cam_pnt_a + cam_pnt_b) / 2
mid_lid_ab = (lid_pnt_a + lid_pnt_b) / 2

mid_lid_on_cam = project_lid_on_img_1(mid_lid_ab, R_final, T, p)

print(torch.FloatTensor(mid_img_ab), mid_lid_on_cam)
e1 = torch.dist(torch.FloatTensor(mid_img_ab), mid_lid_on_cam)
# e1.backward()

def func(p1,p2):
    return torch.add(torch.pow(p1[0]-p2[0],2),torch.pow(p1[1]-p2[1],2))

err = autograd.Variable(func(autograd.Variable(torch.FloatTensor(mid_img_ab)),autograd.Variable(mid_lid_on_cam)))
err.backward()

In [ ]:
a = torch.from_numpy(lid_pnt_b)
print(type(a))
b = torch.cat((a.float() ,torch.FloatTensor([1.])))


In [ ]:
torch.FloatTensor([0,0,0]).view(1,3)

In [ ]:
torch.pow((mid_lid_on_cam[0]-mid_lid_on_cam[1])**2,0.5)

In [ ]:
np.zeros(3)